In [8]:
import numpy as np 
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import MobileNet
from sklearn.metrics import  confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action = "ignore", category = FutureWarning)
%matplotlib inline

In [9]:
# creating directory
import os
try:
    os.mkdir("antispoofing_dataset")
    os.mkdir("antispoofing_dataset/test")
    os.mkdir("antispoofing_dataset/train")
    os.mkdir("antispoofing_dataset/valid")
    os.mkdir("antispoofing_dataset/train/real")
    os.mkdir("antispoofing_dataset/train/spoof")
    os.mkdir("antispoofing_dataset/train/animals")
    os.mkdir("antispoofing_dataset/test/real")
    os.mkdir("antispoofing_dataset/test/spoof")
    os.mkdir("antispoofing_dataset/test/animals")
    os.mkdir("antispoofing_dataset/valid/real")
    os.mkdir("antispoofing_dataset/valid/spoof")
    os.mkdir("antispoofing_dataset/valid/animals")
except FileExistsError:
    print("File Already exist")
    

    
    

File Already exist


In [10]:
dataset_path = "final_antispoofing"
test_path = "antispoofing_dataset/test"
train_path = "antispoofing_dataset/train"
valid_path = "antispoofing_dataset/valid"
animals_path = "antispoofing_dataset/animals"

In [73]:

def train_test_split(doct):
    for i_split in os.listdir(doct):
        path_i_split = os.path.join(doct, i_split)
        for cat in os.listdir(path_i_split):
            path_cat = os.path.join(path_i_split, cat)
            for sub in os.listdir(path_cat):
                path_sub = os.path.join(path_cat, sub)
                for img in os.listdir(path_sub):
                    if i_split == 'train':
                        shutil.copy(os.path.join(path_sub, img), os.path.join(train_path, cat, img))
                    else:
                        shutil.copy(os.path.join(path_sub, img), os.path.join(test_path, cat, img))
                    
                        
train_test_split(doct=dataset_path)

KeyboardInterrupt: 

In [5]:
real_samples = random.sample(os.listdir(f'antispoofing_dataset/test/real'), 300)
for i in real_samples:
    shutil.move(f'antispoofing_dataset/test/real/{i}', f'antispoofing_dataset/valid/real')

spoof_samples = random.sample(os.listdir(f'antispoofing_dataset/test/spoof'), 300)
for k in spoof_samples:
    shutil.move(f'antispoofing_dataset/test/spoof/{k}', f'antispoofing_dataset/valid/spoof')
    
animal_samples_train = random.sample(os.listdir(f'test1'), 3000)
for j in animal_samples_train:
    shutil.move(f'test1/{j}', f'antispoofing_dataset/train/animals')

animal_samples_test = random.sample(os.listdir(f'test1'), 1000)
for m in animal_samples_test:
    shutil.move(f'test1/{m}', f'antispoofing_dataset/test/animals')

animal_samples_valid = random.sample(os.listdir(f'test1'), 1000)
for n in animal_samples_valid:
    shutil.move(f'test1/{n}', f'antispoofing_dataset/valid/animals')  


NameError: name 'random' is not defined

In [11]:
from IPython.display import Image, display
import PIL
from pathlib import Path
from PIL import UnidentifiedImageError

imgs = os.listdir(f'antispoofing_dataset/test/real')
for i in imgs:
    try:
        img = PIL.Image.open(i)
    except UnidentifiedImageError:
            print(i)


FileNotFoundError: [Errno 2] No such file or directory: '20201110_113245.jpg'

In [12]:
mobile = tf.keras.applications.mobilenet.MobileNet()

In [13]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(224,224), batch_size=25, color_mode="rgb")

valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224,224), batch_size=25, color_mode="rgb")

test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(224,224), batch_size=25, color_mode="rgb", shuffle = False)

Found 7220 images belonging to 3 classes.
Found 1600 images belonging to 3 classes.
Found 1951 images belonging to 3 classes.


In [14]:
imgs, labels = next(train_batches)

In [15]:
test_batches

In [16]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
        ax.axis("off")
    plt.tight_layout()
    plt.show()

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

[[0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [17]:
plotImages(imgs)
print(labels)

In [18]:
mobile = tf.keras.applications.mobilenet.MobileNet()
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [19]:
mobile = MobileNet(weights= "imagenet", include_top = False,   input_tensor=Input(shape=(224,224,3)))
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [20]:
output = Flatten()(mobile.output)
output = Dropout(0.3)(output)
output = Dense(units = 8, activation = "relu")(output)
prediction = Dense(3, activation= "sigmoid")(output)

In [21]:
model = Model(inputs=mobile.input, outputs=prediction)


In [22]:
for layer in model.layers[:-18]:
    layer.trainable = False

In [23]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                      

In [24]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [25]:
model.fit(x=train_batches,
            validation_data=valid_batches,
            epochs=10,
            verbose=2)

Epoch 1/10
289/289 - 190s - loss: 0.0637 - accuracy: 0.9701 - val_loss: 4.2368e-04 - val_accuracy: 1.0000 - 190s/epoch - 658ms/step
Epoch 2/10
289/289 - 194s - loss: 0.0015 - accuracy: 0.9999 - val_loss: 1.3636e-04 - val_accuracy: 1.0000 - 194s/epoch - 671ms/step
Epoch 3/10
289/289 - 207s - loss: 6.9636e-04 - accuracy: 1.0000 - val_loss: 6.4982e-05 - val_accuracy: 1.0000 - 207s/epoch - 715ms/step
Epoch 4/10
289/289 - 216s - loss: 4.0327e-04 - accuracy: 1.0000 - val_loss: 4.1497e-05 - val_accuracy: 1.0000 - 216s/epoch - 747ms/step
Epoch 5/10


In [ ]:
test_labels = test_batches.classes

In [60]:
test_batches.class_indices

{'animals': 0, 'real': 1, 'spoof': 2}

In [61]:
predict = model.predict(test_batches, verbose = 0)

In [62]:
for i in predict:
    print(i)

[9.9999809e-01 7.0137190e-08 5.1484905e-09]
[9.999994e-01 7.992883e-09 7.776818e-08]
[1.0000000e+00 3.2247379e-13 4.9347304e-06]
[9.9999976e-01 1.2461796e-09 3.1075679e-08]
[9.9999952e-01 4.9301838e-09 1.2067898e-07]
[1.0000000e+00 1.9010804e-10 1.7565421e-07]
[9.9999875e-01 1.9900423e-08 1.6962740e-07]
[9.9999940e-01 4.3677527e-08 5.9073573e-09]
[1.0000000e+00 2.1162858e-11 1.0589415e-08]
[9.9999952e-01 8.7116196e-11 1.7934045e-06]
[1.0000000e+00 9.1284626e-11 2.5100222e-08]
[9.9999976e-01 2.7188619e-08 1.2462603e-09]
[9.9999976e-01 2.1688278e-08 4.1457096e-10]
[9.9999970e-01 2.2425873e-08 9.1198876e-10]
[9.9999976e-01 1.0928352e-08 4.6633635e-09]
[1.0000000e+00 2.9562304e-09 2.0592654e-11]
[9.9999940e-01 2.2932610e-08 1.1193571e-08]
[9.9999106e-01 1.7629436e-06 2.9784448e-09]
[1.000000e+00 2.077156e-10 8.229748e-08]
[9.9999964e-01 1.9813049e-08 2.5235005e-09]
[9.9996758e-01 1.3620726e-06 4.1483911e-10]
[9.9999762e-01 3.7146191e-07 1.0134349e-08]
[1.0000000e+00 1.1153437e-09 4.2469885

In [1]:
#round_predictions = np.argmax(predict, axis = -1)
round_predictions = np.round(predict)

NameError: name 'np' is not defined

In [64]:
for i in round_predictions:
    print(i)

[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]

In [ ]:
import os.path
if os.path.isfile("models/face_recognition_model.h5") is False:
    model.save("models/face_recognition_model.h5")

In [65]:
def startCamera(self):
        print("Starting camera")
        if not self.hasCamera:
            # initialize the video stream and allow the camera sensor to warmup
            self.hasCamera = True
            self.vs = VideoStream(src=0).start()
            time.sleep(2.0)

In [66]:
#frame = self.vs.read()
#frame = imutils.resize(frame, width=400)

NameError: name 'self' is not defined